## Flask-Security
powerful extension that allows you to implement common security features like authentication, user registration, role management, and more. This tutorial will guide you through each of the listed features:

- Authentication (via session, Basic HTTP, or token)
- User registration (optional)
- Role and Permission management
- Account activation (via email confirmation) (optional)
- Password management (recovery and resetting) (optional)
- Two-factor authentication (optional)
- WebAuthn Support (optional)
- Use ‘social’/Oauth for authentication (e.g. google, github, ..) (optional)
- Change email (optional)
- Login tracking (optional)
- JSON/Ajax Support

### Prerequisites
Before starting, ensure you have the following installed:

- Flask
- Flask-SQLAlchemy
- Flask-Security
- Flask-Bcrypt
- Flask-Migrate
- Flask-Mail (for email support)

In [ ]:
# Installation Prerequisites
!pip install Flask-SQLAlchemy Flask-WTF Flask-Bcrypt Flask-Migrate Flask-Mail Flask-OAuthlib

In [ ]:
# Installation
!pip install Flask-Security

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_security import Security, SQLAlchemyUserDatastore
from flask_bcrypt import Bcrypt
from flask_mail import Mail

# Initialize extensions
db = SQLAlchemy()
migrate = Migrate()
bcrypt = Bcrypt()
mail = Mail()

# Flask-Security setup
security = Security()

def create_app(config_class='config.Config'):
    app = Flask(__name__)
    app.config.from_object(config_class)

    # Initialize extensions
    db.init_app(app)
    migrate.init_app(app, db)
    bcrypt.init_app(app)
    mail.init_app(app)
    
    from models import User,Role
    
    security.init_app(app, SQLAlchemyUserDatastore(db, User, Role))

    from . import routes
    app.register_blueprint(routes.bp)

    return app


### Role and Permission Management
makes it easy to manage roles and permissions within your application. This allows you to create different user roles (e.g., Admin, User) and restrict access to certain parts of the application based on the user's role.

#### Define Roles
Flask-Security expects certain models and relationships for role-based access control (**RBAC**).

- **1. Role Model**:
    - The Role model defines a role (e.g., Admin, User, Moderator, etc.).
    - Use Flask-Security's RoleMixin to automatically add role-specific behavior.

In [ ]:
from flask_security import RoleMixin, UserMixin
from . import db

class Role(db.Model, RoleMixin):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), unique=True)  # 'admin', 'user', etc.
    description = db.Column(db.String(255))  # Optional description

- **2.User Model**:
    - The User model represents the users in the system.
    - Use Flask-Security’s UserMixin to add user-specific behavior like checking if the user is authenticated, active, etc.

In [ ]:
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(255), unique=True)
    password = db.Column(db.String(255))
    active = db.Column(db.Boolean())
    # Define many-to-many relationship between User and Role
    roles = db.relationship('Role', secondary='user_roles')

- **3. Association Table (UserRoles)**:
    - This table links users to roles using a many-to-many relationship. It allows one user to have multiple roles.

In [ ]:
class UserRoles(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id', ondelete='CASCADE'))
    role_id = db.Column(db.Integer, db.ForeignKey('role.id', ondelete='CASCADE'))

This setup allows you to define different roles, assign users multiple roles, and use Flask-Security’s decorators (like **@roles_required**) to restrict access to routes based on roles.

- **Why This Model?**
    - **RoleMixin**: Automatically provides useful methods for role management.
    - **UserMixin**: Automatically provides user-related methods like is_authenticated, is_active, and more.
    - **Many-to-many relationship**: A flexible way to handle users with multiple roles, common in RBAC systems.

#### Assign Roles to Users

In [ ]:
from flask_security import SQLAlchemyUserDatastore

# Initialize SQLAlchemyUserDatastore
user_datastore = SQLAlchemyUserDatastore(db, User, Role)

# Assign a role to a user (for example, Admin)
user = user_datastore.get_user(user_id)
user_datastore.add_role_to_user(user, 'admin')
db.session.commit()

#### Protect routes

In [ ]:
from flask_security import roles_required

@app.route('/admin')
@roles_required('admin')  # Only accessible to users with the 'admin' role
def admin():
    return "Welcome, Admin!"

###  Account Activation (via Email Confirmation)
helps verify the user's email before allowing them to use the system. Flask-Security provides built-in support for email confirmation.

#### Enable Account Activation
by adding SECURITY_CONFIRMABLE = True in your config.py.

In [ ]:
# Enable email confirmation for account activation
SECURITY_CONFIRMABLE = True  # Required for email confirmation (account activation)

# Option to send a welcome email after registration
SECURITY_SEND_REGISTER_EMAIL = False  # Set to False to prevent sending a welcome email until account is activated

#### Configure Flask-Mail
Ensure that Flask-Mail is configured to send emails. Add your mail server configuration in config.py.

In [ ]:
# Flask-Mail settings (for sending emails)
MAIL_SERVER = 'smtp.gmail.com'
MAIL_PORT = 587
MAIL_USE_TLS = True
MAIL_USERNAME = 'your_email@gmail.com'
MAIL_PASSWORD = 'your_email_password'
MAIL_DEFAULT_SENDER = 'your_email@gmail.com'  # Sender email address for confirmation emails

#### Handling Registration Logic in Registration Route

In [ ]:
# routes.py
from flask import render_template, redirect, url_for, flash
from flask_security.utils import hash_password, send_confirmation_instructions
from yourapp.models import User  # Import your User model
from yourapp import db  # Your SQLAlchemy instance
from yourapp.forms import RegistrationForm

@app.route('/register', methods=['GET', 'POST'])
def register():
    form = RegistrationForm()
    if form.validate_on_submit():
        # Check if the user already exists
        existing_user = User.query.filter_by(email=form.email.data).first()
        if existing_user:
            flash('A user with that email already exists', 'danger')
            return redirect(url_for('register'))
        
        # Create a new user
        new_user = User(
            email=form.email.data,
            password=hash_password(form.password.data),  # Use hash_password from Flask-Security
            active=False  # New user is not active until they confirm their email
        )
        db.session.add(new_user)
        db.session.commit()

        # Send confirmation email
        send_confirmation_instructions(new_user)  # Flask-Security utility to send the confirmation email

        flash('A confirmation email has been sent. Please check your inbox.', 'info')
        return redirect(url_for('login'))
    
    return render_template('register.html', form=form)


#### Handle Email Confirmation Link (Activation)
Flask-Security has a built-in route for handling confirmation links that users click in their email, so you don’t need to manually create a route for that. Once the user clicks the confirmation link, Flask-Security will automatically:
- Validate the token in the confirmation link.
- Activate the user by setting active=True.

#### Enforce Email Verification
In your views, you can enforce email confirmation using the @confirm_email_required decorator.

In [ ]:
from flask_security import confirm_email_required

@app.route('/dashboard')
@confirm_email_required
def dashboard():
    return "Welcome to your dashboard!"

Once a user registers, they will receive a confirmation email, and their account will only be activated after they click the confirmation link.

### Password Management (Recovery and Resetting)
Flask-Security supports password recovery and resetting, allowing users to reset their password if they forget it. This feature includes sending a recovery email with a reset link.

#### Enable Password Recovery

In [ ]:
SECURITY_RECOVERABLE = True

#### Configure MAIL_SERVER 
configure MAIL_SERVER settings to ensure that Flask can send recovery emails.

In [ ]:
# Flask-Mail settings for sending recovery emails
MAIL_SERVER = 'smtp.gmail.com'  # Use your email provider's SMTP server
MAIL_PORT = 587  # Common port for TLS encryption
MAIL_USE_TLS = True  # Use TLS encryption
MAIL_USERNAME = 'your_email@gmail.com'  # Your email address
MAIL_PASSWORD = 'your_email_password'  # Your email password
MAIL_DEFAULT_SENDER = 'your_email@gmail.com'  # The sender address for recovery emails

####  Create the Password Recovery Form
This form will allow the user to input their email address to request a password reset link.

In [ ]:
# forms.py
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired, Email

class PasswordRecoveryForm(FlaskForm):
    email = StringField('Email', validators=[DataRequired(), Email()])
    submit = SubmitField('Recover Password')

In [ ]:
# views.py
from flask import render_template, redirect, url_for, flash
from .forms import PasswordRecoveryForm
from flask_security import send_reset_password_instructions

@app.route('/recover', methods=['GET', 'POST'])
def recover_password():
    form = PasswordRecoveryForm()
    if form.validate_on_submit():
        email = form.email.data
        # Trigger Flask-Security's email sending
        send_reset_password_instructions(email)
        flash('A password reset email has been sent if the email exists in our system.', 'info')
        return redirect(url_for('login'))
    return render_template('recover.html', form=form)

- The view processes the form submission and triggers Flask-Security's send_reset_password_instructions to send the reset email.
- It displays a success message if the form is successfully submitted.

In [ ]:
<!-- recover.html -->
<form method="POST" action="{{ url_for('recover_password') }}">
    {{ form.hidden_tag() }}  <!-- CSRF protection -->
    <div>
        {{ form.email.label }} 
        {{ form.email() }}  <!-- Email field -->
    </div>
    <div>
        {{ form.submit() }}  <!-- Submit button -->
    </div>
</form>

#### Email Confirmation for Password Recovery
Once the user submits the password recovery form, Flask-Security automatically generates a unique token and sends an email to the user. The email contains a link to the password reset page, which includes the token in the URL.<br>

- Flask-Security manages this email generation and sending based on the configuration you provided (such as MAIL_SERVER, MAIL_USERNAME, etc.).

#### Create the Password Reset Form
This form will allow users to reset their password after they click the reset link in their email.

In [ ]:
# forms.py
from flask_wtf import FlaskForm
from wtforms import PasswordField, SubmitField
from wtforms.validators import DataRequired, EqualTo

class PasswordResetForm(FlaskForm):
    password = PasswordField('New Password', validators=[DataRequired()])
    password_confirm = PasswordField('Confirm New Password', validators=[DataRequired(), EqualTo('password')])
    submit = SubmitField('Reset Password')

In [ ]:
# views.py
from flask import render_template, redirect, url_for, flash, request
from .forms import PasswordResetForm
from flask_security import reset_password

@app.route('/reset/<token>', methods=['GET', 'POST'])
def reset_password_view(token):
    form = PasswordResetForm()
    if form.validate_on_submit():
        new_password = form.password.data
        # Call Flask-Security's reset_password function with the token
        if reset_password(token, new_password):
            flash('Your password has been reset successfully.', 'success')
            return redirect(url_for('login'))
        else:
            flash('The reset token is invalid or expired.', 'danger')
            return redirect(url_for('recover_password'))
    return render_template('reset_password.html', form=form)


<!-- reset_password.html -->
<form method="POST" action="{{ url_for('reset_password_view', token=token) }}">
    {{ form.hidden_tag() }}  <!-- CSRF protection -->
    <div>
        {{ form.password.label }} 
        {{ form.password() }}  <!-- New password field -->
    </div>
    <div>
        {{ form.password_confirm.label }} 
        {{ form.password_confirm() }}  <!-- Confirm password field -->
    </div>
    <div>
        {{ form.submit() }}  <!-- Submit button -->
    </div>
</form>


When the form is submitted, Flask-Security:

- Verifies that the reset token is valid (it was included in the email link).
- Verifies that the new password and password confirmation match.
Hashes the new password and updates the user’s record in the database.


#### Automatic Email Notifications
Flask-Security automatically sends an email when the password is successfully reset, notifying the user that their password has been changed.

#### Additional Notes
- **Token Expiration**: Flask-Security uses tokens for password recovery that expire after a certain period for security reasons. You can control this with the **SECURITY_RESET_SALT** and **SECURITY_RESET_WITHIN** settings.